# Approve and Deploy the Model

The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

As part of SageMaker Pipelines, data scientists can register the model with approved/pending manual approval as part of the CI/CD workflow.

We can also approve the model using the SageMaker Studio UI or programmatically as shown below.

In [1]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=4072407040, available=2023182336, percent=50.3, used=1797156864, free=113033216, active=1854042112, inactive=1844625408, buffers=0, cached=2162216960, shared=966656, slab=159789056)
*******************************************
YOU ARE NOT USING THE CORRECT INSTANCE TYPE
PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge 
*******************************************


In [2]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

In [3]:
%store -r role

# List Pipeline Execution Steps


In [4]:
%store -r pipeline_name

In [5]:
print(pipeline_name)

pipeline-1678413961


In [6]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

Succeeded
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:079002598131:pipeline/pipeline-1678413961/execution/yrq67uomu7xy',
  'PipelineExecutionDisplayName': 'execution-1678413967767',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2023, 3, 10, 2, 6, 7, 683000, tzinfo=tzlocal())}]
CPU times: user 13.8 ms, sys: 1.22 ms, total: 15.1 ms
Wall time: 340 ms


In [7]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

Succeeded


In [8]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:079002598131:pipeline/pipeline-1678413961/execution/yrq67uomu7xy


In [9]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'AttemptCount': 0,
                             'EndTime': datetime.datetime(2023, 3, 10, 2, 33, 14, 413000, tzinfo=tzlocal()),
                             'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:079002598131:model-package/reviews-1678413964/1'}},
                             'StartTime': datetime.datetime(2023, 3, 10, 2, 33, 13, 156000, tzinfo=tzlocal()),
                             'StepName': 'Reviews-RegisterModel',
                             'StepStatus': 'Succeeded'},
                            {'AttemptCount': 0,
                             'EndTime': datetime.datetime(2023, 3, 10, 2, 33, 14, 717000, tzinfo=tzlocal()),
                             'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:079002598131:model/pipelines-yrq67uomu7xy-createmodel-y0hvrinj5k'}},
                             'StartTime': datetime.datetime(2023, 3, 10, 2, 33, 13, 156000, tzinfo=tzlocal()),
                             'StepNa

# View Registered Model

In [10]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "Reviews-RegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break
print(model_package_arn)

arn:aws:sagemaker:us-east-1:079002598131:model-package/reviews-1678413964/1


In [11]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",  # Other options are Rejected and PendingManualApproval
)

# View Created Model

In [12]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

pipeline_model_name = model_arn.split("/")[-1]
print(pipeline_model_name)

arn:aws:sagemaker:us-east-1:079002598131:model/pipelines-yrq67uomu7xy-createmodel-y0hvrinj5k
pipelines-yrq67uomu7xy-createmodel-y0hvrinj5k


# Create Model Endpoint from Model Registry
More details here:  https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-deploy.html


In [13]:
import time

timestamp = int(time.time())

model_from_registry_name = "model-from-registry-{}".format(timestamp)
print("Model from registry name : {}".format(model_from_registry_name))

model_registry_package_container = {
    "ModelPackageName": model_package_arn,
}

Model from registry name : model-from-registry-1678415699


In [14]:
from pprint import pprint

create_model_from_registry_response = sm.create_model(
    ModelName=model_from_registry_name, ExecutionRoleArn=role, PrimaryContainer=model_registry_package_container
)
pprint(create_model_from_registry_response)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:079002598131:model/model-from-registry-1678415699',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '92',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 10 Mar 2023 02:34:59 GMT',
                                      'x-amzn-requestid': '7a923fb2-8154-4b7c-b8b4-02e558b9566b'},
                      'HTTPStatusCode': 200,
                      'RequestId': '7a923fb2-8154-4b7c-b8b4-02e558b9566b',
                      'RetryAttempts': 0}}


In [15]:
model_from_registry_arn = create_model_from_registry_response["ModelArn"]
model_from_registry_arn

'arn:aws:sagemaker:us-east-1:079002598131:model/model-from-registry-1678415699'

#### Note: In this workshop, we are intentionally deploying our model to only 1 instance. The general recommendation is to deploy to 2 or more instances for automatic placement across two AZs for high availability.

In [16]:
endpoint_config_name = "model-from-registry-epc-{}".format(timestamp)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.4xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": pipeline_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

model-from-registry-epc-1678415699


In [17]:
%store -d pipeline_endpoint_name

In [18]:
# Pick up the existing `pipeline_endpoint_name` if it was already created

%store -r pipeline_endpoint_name

no stored variable or alias pipeline_endpoint_name


In [19]:
# try:
#     print("Using existing Pipeline EndpointName: {}".format(pipeline_endpoint_name))
# except NameError:
timestamp = int(time.time())
pipeline_endpoint_name = "model-from-registry-ep-{}".format(timestamp)
print("Created Pipeline EndpointName={}".format(pipeline_endpoint_name))

create_endpoint_response = sm.create_endpoint(
    EndpointName=pipeline_endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

Created Pipeline EndpointName=model-from-registry-ep-1678415700
arn:aws:sagemaker:us-east-1:079002598131:endpoint/model-from-registry-ep-1678415700


In [20]:
%store pipeline_endpoint_name

Stored 'pipeline_endpoint_name' (str)


In [21]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

# _Wait Until the Endpoint is Deployed_
_Note:  This will take a few minutes.  Please be patient._

In [22]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)

CPU times: user 75.9 ms, sys: 15.1 ms, total: 91 ms
Wall time: 2min 31s


# _Wait Until the Endpoint ^^ Above ^^ is Deployed_

# Release Resources

In [23]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}
</script>